## Exploring Views
* In Databricks, **views** act as virtual tables without holding physical data.
* A view is simply a **saved SQL query** against actual tables.
* The underlying SQL query executes **every time** the view is queried.

**View Types**

* Databricks supports **three types of views**:

  * **Stored Views**
  * **Temporary Views**
  * **Global Temporary Views**

In [0]:
USE CATALOG hive_metastore;

DROP TABLE IF EXISTS smartphones;
CREATE TABLE smartphones (
  id INT,
  model STRING,
  brand STRING,
  release_year INT
);

In [0]:
INSERT INTO smartphones
VALUES 
(1, 'iPhone 15 Pro', 'Apple', 2023),
(2, 'Galaxy S23 Ultra', 'Samsung', 2023),
(3, 'Pixel 8 Pro', 'Google', 2023),
(4, 'OnePlus 12', 'OnePlus', 2024),
(5, 'Xperia 1 V', 'Sony', 2023),
(6, 'Galaxy Z Fold5', 'Samsung', 2023),
(7, 'iPhone SE (3rd Gen)', 'Apple', 2022),
(8, 'Pixel 7a', 'Google', 2023),
(9, 'Moto Edge+ (2023)', 'Motorola', 2023),
(10, 'Nothing Phone (2)', 'Nothing', 2023);

In [0]:
SHOW TABLES;

### Stored Views
* Also known simply as **views**, similar to traditional database views.
* Their metadata is **persisted in the database**.
* Created using `CREATE VIEW` followed by `AS <query>`.
* Use `SHOW TABLES` to confirm that the view is saved in the **default database**.


In [0]:
CREATE VIEW view_apple_phones
AS
SELECT *
FROM smartphones
WHERE brand = 'Apple';

Executing the **`SHOW TABLES`** command again confirms that the view has been stored in the **default** database as a persistent object rather than a temporary one.

In [0]:
SHOW TABLES;

Once created, you can query the stored view using a standard SELECT statement,
treating it as if it were a table object

In [0]:
SELECT * FROM view_apple_phones

### Temporary Views
* **Bound to the Spark session**; automatically dropped when the session ends.
* Useful for **temporary data manipulations** or analysis.
* Created with the `TEMP` or `TEMPORARY` keyword.

  **Example**:

  `CREATE TEMP VIEW view_name AS <query>;`
  
  **Lifespan ends** when:

  * A new notebook is opened.
  * The notebook detaches and reattaches to a cluster.
  * The Python interpreter restarts.
  * The cluster restarts.

In [0]:
CREATE TEMP VIEW temp_view_smartphone_brands
AS
SELECT DISTINCT brand
FROM smartphones;

In [0]:
SHOW TABLES

In [0]:
SELECT * FROM temp_view_smartphone_brands;

### Global Temporary Views

* Behave like temporary views but **persist across sessions within the same cluster**.
* Created using `GLOBAL TEMP`.

  **Example**:

  `CREATE GLOBAL TEMP VIEW view_name`<br>
  `AS <query>;`<br>

* Stored in the **`global_temp` database**.
* To query them, specify the database qualifier:

  `SELECT * FROM global_temp.view_name;`

* Not listed by default in `SHOW TABLES`; instead, use:

  `SHOW TABLES IN global_temp;`

In [0]:
CREATE GLOBAL TEMP VIEW global_temp_view_recent_smartphones
AS
SELECT *
FROM smartphones
WHERE release_year >= 2023;

In [0]:
SHOW TABLES;

In [0]:
SHOW TABLES in global_temp

In [0]:
-- To query the global temp view:
SELECT * FROM global_temp.global_temp_view_recent_smartphones;

### Dropping Views
Drop our stored view by running the DROP VIEW command, like in standard SQL:

In [0]:
DROP VIEW view_apple_phones

If you prefer not to wait for the session to end or the cluster to shut down, you can manually remove temporary views by using the **`DROP VIEW`** command, for example:

In [0]:
SELECT * FROM temp_view_smartphone_brands;

In [0]:
DROP VIEW temp_view_smartphone_brands;
DROP VIEW global_temp.global_temp_view_recent_smartphones;

### Comparison of View Types

| **Aspect**           | **Stored View**                             | **Temporary View**                              | **Global Temporary View**                              |
| -------------------- | ------------------------------------------- | ----------------------------------------------- | ------------------------------------------------------ |
| **Creation Syntax**  | `CREATE VIEW`                               | `CREATE TEMP VIEW`                              | `CREATE GLOBAL TEMP VIEW`                              |
| **Persistence**      | Saved in the **database metadata**          | Exists **only in the current Spark session**    | Exists across sessions, but **tied to the cluster**    |
| **Accessibility**    | Accessible across **sessions and clusters** | Accessible **only within the session**          | Accessible across sessions **within the same cluster** |
| **Storage Location** | Default database                            | Session memory                                  | `global_temp` database                                 |
| **Lifetime**         | Until explicitly **dropped**                | **Dropped automatically** when the session ends | **Dropped automatically** when the cluster stops       |
| **Manual Deletion**  | `DROP VIEW view_name;`                      | `DROP VIEW view_name;` (optional)               | `DROP VIEW global_temp.view_name;` (optional)          |
| **Usage Example**    | `SELECT * FROM view_name;`                  | `SELECT * FROM view_name;`                      | `SELECT * FROM global_temp.view_name;`                 |


